## Speech recognition
### Training the network using a training set. Demonstrating the ability of the network to recognize words not in the training set and at the end, reporting the accuracy of the model.

### Preparing the data and importing functions from preproces.py

In [4]:
%load_ext autoreload
%autoreload 2

from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical

# Second dimension of the feature is dim2
feature_dim_2 = 11

# Save data to array file first
save_data_to_array(max_len=feature_dim_2)

# # Loading train set and test set
X_train, X_test, y_train, y_test = get_train_test()

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 50
batch_size = 100
verbose = 1
num_classes = 5

# Reshaping to perform 2D convolution
X_train = X_train.reshape(X_train.shape[0], feature_dim_1, feature_dim_2, channel)
X_test = X_test.reshape(X_test.shape[0], feature_dim_1, feature_dim_2, channel)

y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving vectors of label - 'yes': 100%|█████| 2377/2377 [00:55<00:00, 43.05it/s]


In [5]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

# Building The Model Then Training it

In [6]:
model = get_model()
model.fit(X_train, y_train_hot, batch_size=batch_size, epochs=epochs, verbose=verbose, validation_data=(X_test, y_test_hot))

Train on 5586 samples, validate on 3725 samples
Epoch 1/50
5586/5586 [==============================] - ETA: 2:01 - loss: 4.9832 - acc: 0.170 - ETA: 1:05 - loss: 6.6667 - acc: 0.185 - ETA: 46s - loss: 6.5088 - acc: 0.176 - ETA: 36s - loss: 5.7144 - acc: 0.18 - ETA: 31s - loss: 5.0730 - acc: 0.19 - ETA: 27s - loss: 4.5858 - acc: 0.20 - ETA: 25s - loss: 4.1977 - acc: 0.21 - ETA: 22s - loss: 3.9210 - acc: 0.20 - ETA: 21s - loss: 3.6740 - acc: 0.21 - ETA: 19s - loss: 3.4888 - acc: 0.21 - ETA: 18s - loss: 3.3287 - acc: 0.21 - ETA: 17s - loss: 3.1868 - acc: 0.22 - ETA: 16s - loss: 3.0742 - acc: 0.23 - ETA: 15s - loss: 2.9773 - acc: 0.23 - ETA: 14s - loss: 2.8878 - acc: 0.23 - ETA: 14s - loss: 2.8153 - acc: 0.23 - ETA: 13s - loss: 2.7473 - acc: 0.23 - ETA: 13s - loss: 2.6906 - acc: 0.23 - ETA: 12s - loss: 2.6331 - acc: 0.23 - ETA: 11s - loss: 2.5771 - acc: 0.23 - ETA: 11s - loss: 2.5309 - acc: 0.23 - ETA: 10s - loss: 2.4838 - acc: 0.24 - ETA: 10s - loss: 2.4454 - acc: 0.24 - ETA: 9s - loss: 2

## Prediction: trying with the word "dog"

In [25]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/dog/0b09edd3_nohash_1.wav', model=model))

dog


## Second try with the word "dog"

In [27]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/dog/0ff728b5_nohash_0.wav', model=model))

dog


## Trying with the word "yes"

In [23]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/yes/01bb6a2a_nohash_2.wav', model=model))


yes


## Second try with the word "yes"

In [24]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/yes/0f7dc557_nohash_1.wav', model=model))

yes


## Checking with the word "down" not included in the training set:

In [29]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/down/98447c43_nohash_0.wav', model=model))

dog


We can see that the network identified the word as "dog". 

## Checking with the word "house" not included in the training set:

In [31]:
print(predict('/users/Victor/documents/Master/CSCI E-89 Deep Learning/Assignments/Assignment 12/DeadSimpleSpeechRecognizer-master/data/house/0d393936_nohash_0.wav', model=model))

down


We can see that the network identified the word as "down".